In [2]:
import sys
import os
import random
from collections import Counter
import pandas as pd

class FriendNetworkGenerator:
    def __init__(self):
        self.user_friend = dict()
        self.item_user = dict()
        self.user_item = dict()
        self.user_recitems = dict()
        self.nuser_recitems = dict()
        self.userlist = []
        self.test2days_item_user = dict()
        self.test2days_user_item = dict()
        self.test4days_item_user = dict()
        self.test4days_user_item = dict()
        self.test10days_item_user = dict()
        self.test10days_user_item = dict()
        self.selected_user_items = dict()

    def read_data(self, dirpath):
        with open(dirpath + "/item_user.txt") as iufile:  # list of all items purchased by users
            for line in iufile:
                toks = line.strip().split("\t")
                if len(toks) == 2:
                    i, u = toks[0], toks[1]
                    if i not in self.item_user:
                        self.item_user[i] = []
                    self.item_user[i].append(u)   # {itemid1: [uid1,uid2,...], itemid2: [uid2,uid3,....]} = dict of list of users purchasing each item 
                    if u not in self.user_item:
                        self.user_item[u] = []
                    self.user_item[u].append(i)   # {uid1: [itemid1,itemid2,...], uid2: [itemd2,itemd3,....]} = dict of list of items purchased by each user

        with open(dirpath + "/test2days_item_user.txt") as iufile:  # list of all items purchased by users in 2 days of testing period
            for line in iufile:
                toks = line.strip().split("\t")
                if len(toks) == 2:
                    i, u = toks[0], toks[1]
                    if i not in self.test2days_item_user:
                        self.test2days_item_user[i] = []
                    self.test2days_item_user[i].append(u)   # {itemid1: [uid1,uid2,...], itemid2: [uid2,uid3,....]} = dict of list of users purchasing each item 
                    if u not in self.test2days_user_item:
                        self.test2days_user_item[u] = []
                    self.test2days_user_item[u].append(i)   # {uid1: [itemid1,itemid2,...], uid2: [itemd2,itemd3,....]} = dict of list of items purchased by each user

        with open(dirpath + "/test4days_item_user.txt") as iufile:  # list of all items purchased by users in 4 days of testing period
            for line in iufile:
                toks = line.strip().split("\t")
                if len(toks) == 2:
                    i, u = toks[0], toks[1]
                    if i not in self.test4days_item_user:
                        self.test4days_item_user[i] = []
                    self.test4days_item_user[i].append(u)   # {itemid1: [uid1,uid2,...], itemid2: [uid2,uid3,....]} = dict of list of users purchasing each item 
                    if u not in self.test4days_user_item:
                        self.test4days_user_item[u] = []
                    self.test4days_user_item[u].append(i)   # {uid1: [itemid1,itemid2,...], uid2: [itemd2,itemd3,....]} = dict of list of items purchased by each user

        with open(dirpath + "/test10days_item_user.txt") as iufile:  # list of all items purchased by users in 10 days of testing period
            for line in iufile:
                toks = line.strip().split("\t")
                if len(toks) == 2:
                    i, u = toks[0], toks[1]
                    if i not in self.test10days_item_user:
                        self.test10days_item_user[i] = []
                    self.test10days_item_user[i].append(u)   # {itemid1: [uid1,uid2,...], itemid2: [uid2,uid3,....]} = dict of list of users purchasing each item 
                    if u not in self.test10days_user_item:
                        self.test10days_user_item[u] = []
                    self.test10days_user_item[u].append(i)   # {uid1: [itemid1,itemid2,...], uid2: [itemd2,itemd3,....]} = dict of list of items purchased by each user

        
        with open(dirpath + "/user_friend.txt") as uffile:  # list of all friendship linkages
            for line in uffile:
                toks = line.strip().split("\t")
                if len(toks) == 2:
                    u, f = toks[0], toks[1]
                    if u not in self.user_friend:
                        self.user_friend[u] = []
                    self.user_friend[u].append(f)   # {uid1: [friendid1,friendid2,...], uid2: [friendid2,friendid3,....]} = dict of each person's friends
        
        with open(dirpath + "/selected_user.txt") as sufile:  # pull the randomly selected users
            for user in sufile:
                self.userlist.append(user.strip())
                
    def get_friend_item_recommendations(self):
        print("#Number of users in consideration set:", len(self.user_friend))
        print("\n")
        for user in self.user_friend:                                # iterating through users
            friends = self.user_friend[user]                         # fetching friends of the iterated user
            for friend in friends:                                   # iterating through the friends of the user
                items = self.user_item.get(friend,[])                # fetching items bought by the particular iterated friend
                if len(items) > 0:
                    for item in items:                               # iterating through items of the friend
                        if item not in self.user_item.get(user,[]):  # if the item purchased by friend has not already been purchased by the user, recommend the item to him
                            if user not in self.user_recitems:
                                self.user_recitems[user] = []
                            if item not in self.user_recitems[user]: # to make sure duplicate items don't get recommended from multiple friends
                                self.user_recitems[user].append(item)    # {uid1: [itemid1,itemid2,...], uid2: [itemd2,itemd3,....]} = dict of list of users with recommended items that their friends purchased 
        sample_items = {k: self.user_recitems[k] for k in list(self.user_recitems)[:20]}
        print(sample_items)
        print("\n")
        print("#Number of users in training data for whom items are recommended:", len(self.user_recitems))
        print("\n")
        
    def get_item_recommendations_selected_users(self):
        for user in self.userlist:
            self.selected_user_items[user] = self.user_recitems.get(user,[])
        selected_user_items = {k: self.selected_user_items[k] for k in list(self.selected_user_items)[:20]}
        print(selected_user_items)
        print("\n")
        users_with_items = [user for user in self.selected_user_items if len(self.selected_user_items[user]) > 0]
        print("#Users in the random set for whom items are recommended based on training data:", len(users_with_items))
        print("\n")
        
    def get_accuracy_score(self):
        count_2days = count_4days = count_10days = 0
        for user in self.selected_user_items:
            recommended_items = set(self.selected_user_items[user])
            new_items_2days = set(self.test2days_user_item.get(user,[]))
            new_items_4days = set(self.test4days_user_item.get(user,[]))
            new_items_10days = set(self.test10days_user_item.get(user,[]))
            if len(recommended_items & new_items_2days) > 0:
                count_2days += 1
            if len(recommended_items & new_items_4days) > 0:
                count_4days += 1
            if len(recommended_items & new_items_10days) > 0:
                count_10days += 1
        print("Accuracy for Friendship-based Recommender System")
        print("For a 2-day testing period:", count_2days/10000)
        print("For a 4-day testing period:", count_4days/10000)
        print("For a 10-day testing period:", count_10days/10000)
        
    def get_alternate_selected_user_item_recommendations(self):
        print("\nAlternate Approach\n")
        print("#Number of users randomly selected:", len(self.userlist))
        print("\n")
        for user in self.userlist:                                        
            friends = self.user_friend.get(user,[])                  
            fitems = set()
            for friend in friends:                                        
                fritems = self.user_item.get(friend,[])                  
                if len(fritems) > 0:
                    for item in fritems:                                    
                        fitems.add(item)
            uitems = self.user_item.get(user,[])
            if user not in self.nuser_recitems:
                self.nuser_recitems[user] = []
            for item in fitems:
                if item not in uitems:                                    
                    self.nuser_recitems[user].append(item)                 
        nsample_items = {k: self.nuser_recitems[k] for k in list(self.nuser_recitems)[:20]}
        print(nsample_items)
        print("\n")
        users_with_items = [user for user in self.nuser_recitems if len(self.nuser_recitems[user]) > 0]
        print("#Users in the random set for whom items are recommended based on training data:", len(users_with_items))
        print("\n")
        
    def get_alternate_accuracy_score(self):
        count_2days = count_4days = count_10days = 0
        for user in self.nuser_recitems:
            recommended_items = set(self.nuser_recitems[user])
            new_items_2days = set(self.test2days_user_item.get(user,[]))
            new_items_4days = set(self.test4days_user_item.get(user,[]))
            new_items_10days = set(self.test10days_user_item.get(user,[]))
            if len(recommended_items & new_items_2days) > 0:
                count_2days += 1
            if len(recommended_items & new_items_4days) > 0:
                count_4days += 1
            if len(recommended_items & new_items_10days) > 0:
                count_10days += 1
        print("Accuracy for Friendship-based Recommender System")
        print("For a 2-day testing period:", count_2days/10000)
        print("For a 4-day testing period:", count_4days/10000)
        print("For a 10-day testing period:", count_10days/10000)

                            
dirpath = 'C:/Vidit/PhD/RA Work/KZ - RA/Gaming Project/data/Files'


def main():
    fng = FriendNetworkGenerator()
    fng.read_data(dirpath)
    fng.get_friend_item_recommendations()
    fng.get_item_recommendations_selected_users()
    fng.get_accuracy_score()
    fng.get_alternate_selected_user_item_recommendations()
    fng.get_alternate_accuracy_score()
    print("\nSuccess")


if __name__ == "__main__":
    main()

#Number of users in consideration set: 1302842


{'0': ['lolsam002', 'loltw005', 'loltw003', 'loltw004', 'package_20', 'RP003', 'loltw001', 'package_18', '15327', '276', '9000', '9002', 'RP001', 'RP002', '244', '95', 'HONLUCK1', 'package_4', 'lolvn002', '2', '223', '245', '26', '89', '205', '57', 'fo3vn006', 'fo3vn007', '15279', '193', '145', '151', '159', '166', '202', '31', '33', '43', '9001', '138', '280', 'lolsam003', 'lolsam006', 'fo3sam001', '239', '34', 'RP004', 'RP005', '100003037', '49', 'package_19', 'package_2', 'tw_19_2', 'lolsam001', 'lolsam004', '15054'], '42465': ['package_19', 'lolvn003'], '42569': ['lolvn002', 'lolvn003', 'lolvn001'], '43543': ['loltw001'], '43639': ['145', '151', '2', '239', '43', '57', '9000', '9002', 'fo3sam001', 'fo3sam003', '25', '205', '230', '237', '12', '31', '34', '157', '212', '33', '138', '166', '159', '188', '95', '192', '200', '202', '228', '280', '14', '61', '76', '235', '207', '136', '175', '223', '244'], '43791': ['lolvn001', 'lolvn003'

In [1]:
import sys
import os
import random
from collections import Counter
import pandas as pd

class FriendNetworkGenerator:
    def __init__(self):
        self.user_friend = dict()
        self.item_user = dict()
        self.user_item = dict()
        self.userlist = []
        self.test2days_item_user = dict()
        self.test2days_user_item = dict()
        self.test4days_item_user = dict()
        self.test4days_user_item = dict()
        self.test10days_item_user = dict()
        self.test10days_user_item = dict()

    def read_data(self, dirpath):
        with open(dirpath + "/item_user.txt") as iufile:  # list of all items purchased by users
            for line in iufile:
                toks = line.strip().split("\t")
                if len(toks) == 2:
                    i, u = toks[0], toks[1]
                    if i not in self.item_user:
                        self.item_user[i] = []
                    self.item_user[i].append(u)   # {itemid1: [uid1,uid2,...], itemid2: [uid2,uid3,....]} = dict of list of users purchasing each item 
                    if u not in self.user_item:
                        self.user_item[u] = []
                    self.user_item[u].append(i)   # {uid1: [itemid1,itemid2,...], uid2: [itemd2,itemd3,....]} = dict of list of items purchased by each user

        with open(dirpath + "/test2days_item_user.txt") as iufile:  # list of all items purchased by users in 2 days of testing period
            for line in iufile:
                toks = line.strip().split("\t")
                if len(toks) == 2:
                    i, u = toks[0], toks[1]
                    if i not in self.test2days_item_user:
                        self.test2days_item_user[i] = []
                    self.test2days_item_user[i].append(u)   # {itemid1: [uid1,uid2,...], itemid2: [uid2,uid3,....]} = dict of list of users purchasing each item 
                    if u not in self.test2days_user_item:
                        self.test2days_user_item[u] = []
                    self.test2days_user_item[u].append(i)   # {uid1: [itemid1,itemid2,...], uid2: [itemd2,itemd3,....]} = dict of list of items purchased by each user

        with open(dirpath + "/test4days_item_user.txt") as iufile:  # list of all items purchased by users in 4 days of testing period
            for line in iufile:
                toks = line.strip().split("\t")
                if len(toks) == 2:
                    i, u = toks[0], toks[1]
                    if i not in self.test4days_item_user:
                        self.test4days_item_user[i] = []
                    self.test4days_item_user[i].append(u)   # {itemid1: [uid1,uid2,...], itemid2: [uid2,uid3,....]} = dict of list of users purchasing each item 
                    if u not in self.test4days_user_item:
                        self.test4days_user_item[u] = []
                    self.test4days_user_item[u].append(i)   # {uid1: [itemid1,itemid2,...], uid2: [itemd2,itemd3,....]} = dict of list of items purchased by each user

        with open(dirpath + "/test10days_item_user.txt") as iufile:  # list of all items purchased by users in 10 days of testing period
            for line in iufile:
                toks = line.strip().split("\t")
                if len(toks) == 2:
                    i, u = toks[0], toks[1]
                    if i not in self.test10days_item_user:
                        self.test10days_item_user[i] = []
                    self.test10days_item_user[i].append(u)   # {itemid1: [uid1,uid2,...], itemid2: [uid2,uid3,....]} = dict of list of users purchasing each item 
                    if u not in self.test10days_user_item:
                        self.test10days_user_item[u] = []
                    self.test10days_user_item[u].append(i)   # {uid1: [itemid1,itemid2,...], uid2: [itemd2,itemd3,....]} = dict of list of items purchased by each user

        
        with open(dirpath + "/user_friend.txt") as uffile:  # list of all friendship linkages
            for line in uffile:
                toks = line.strip().split("\t")
                if len(toks) == 2:
                    u, f = toks[0], toks[1]
                    if u not in self.user_friend:
                        self.user_friend[u] = []
                    self.user_friend[u].append(f)   # {uid1: [friendid1,friendid2,...], uid2: [friendid2,friendid3,....]} = dict of each person's friends
        
        with open(dirpath + "/selected_user.txt") as sufile:  # pull the randomly selected users
            for user in sufile:
                self.userlist.append(user.strip())
                
    def get_friend_item_recommendations(self):
        top_5_user_recitems = dict()
        top_10_user_recitems = dict()
        top_15_user_recitems = dict()
        top_20_user_recitems = dict()
        print("\nMaking Recommendations\n")
        print("#Number of users randomly selected:", len(self.userlist))
        for user1 in self.userlist:                                                                                
            self.user_simscore = {}
            self.user_rec = {}
            items1 = self.user_item.get(user1,[])                                                               
            friends1 = set(self.user_friend.get(user1,[]))
            if len(friends1) > 0:
                for friend in friends1:
                    items2 = self.user_item.get(friend,[])
                    if len(items2) > 0:
                        friends2 = set(self.user_friend.get(friend,[]))                                                                
                        mutual_friends = len(friends1 & friends2)
                        total_friends = len(friends1 | friends2)
                        friend_similarity_score = mutual_friends/total_friends
                        rec_items = [item for item in items2 if item not in items1]                                 
                        self.user_rec[friend] = rec_items
                        self.user_simscore[friend] = friend_similarity_score
                self.sorted_user_simscore = sorted(self.user_simscore.items(), key=lambda x: x[1], reverse = True)
                top_5_user_recitems[user1] = self.get_top_k_user_items(k=5)
                top_10_user_recitems[user1] = self.get_top_k_user_items(k=10)
                top_15_user_recitems[user1] = self.get_top_k_user_items(k=15)
                top_20_user_recitems[user1] = self.get_top_k_user_items(k=20)
        print("\n")
        print("Accuracy for Friendship-based Recommender System based on Top-5 similar friends")
        self.get_accuracy_score(user_recitems = top_5_user_recitems)
        print("\n")
        print("Accuracy for Friendship-based Recommender System based on Top-10 similar friends")
        self.get_accuracy_score(user_recitems = top_10_user_recitems)
        print("\n")
        print("Accuracy for Friendship-based Recommender System based on Top-15 similar friends")
        self.get_accuracy_score(user_recitems = top_15_user_recitems)   
        print("\n")
        print("Accuracy for Friendship-based Recommender System based on Top-20 similar friends")
        self.get_accuracy_score(user_recitems = top_20_user_recitems)
        
    def get_top_k_user_items(self, k):
        top_k_users = [user for user,_ in self.sorted_user_simscore[0:k]]   # getting the top k most similar users to the focal user
        top_k_users_items = set()                                           # now we fetch the recommended items for these top users
        for user in top_k_users:
            items = self.user_rec[user]
            for item in items:
                top_k_users_items.add(item)
        return list(top_k_users_items)      
        
    def get_accuracy_score(self, user_recitems):
        count_2days = count_4days = count_10days = 0
        for user in self.userlist:
            recommended_items = set(user_recitems.get(user,[]))
            new_items_2days = set(self.test2days_user_item.get(user,[]))
            new_items_4days = set(self.test4days_user_item.get(user,[]))
            new_items_10days = set(self.test10days_user_item.get(user,[]))
            if len(recommended_items & new_items_2days) > 0:
                count_2days += 1
            if len(recommended_items & new_items_4days) > 0:
                count_4days += 1
            if len(recommended_items & new_items_10days) > 0:
                count_10days += 1
        print("For a 2-day testing period:", count_2days/10000)
        print("For a 4-day testing period:", count_4days/10000)
        print("For a 10-day testing period:", count_10days/10000)

                            
dirpath = 'C:/Vidit/PhD/RA Work/KZ - RA/Gaming Project/data/Files'


def main():
    fng = FriendNetworkGenerator()
    fng.read_data(dirpath)
    fng.get_friend_item_recommendations()
    print("\nSuccess")


if __name__ == "__main__":
    main()


Making Recommendations

#Number of users randomly selected: 10000


Accuracy for Friendship-based Recommender System based on Top-5 similar friends
For a 2-day testing period: 0.0027
For a 4-day testing period: 0.0063
For a 10-day testing period: 0.0132


Accuracy for Friendship-based Recommender System based on Top-10 similar friends
For a 2-day testing period: 0.0027
For a 4-day testing period: 0.0063
For a 10-day testing period: 0.0132


Accuracy for Friendship-based Recommender System based on Top-15 similar friends
For a 2-day testing period: 0.0027
For a 4-day testing period: 0.0063
For a 10-day testing period: 0.0132


Accuracy for Friendship-based Recommender System based on Top-20 similar friends
For a 2-day testing period: 0.0027
For a 4-day testing period: 0.0063
For a 10-day testing period: 0.0132

Success


In [1]:
class FriendNetworkGenerator:
    def __init__(self):
        self.user_friend = dict()
        self.item_user = dict()
        self.user_item = dict()
        self.userlist = []
        self.test2days_item_user = dict()
        self.test2days_user_item = dict()
        self.test4days_item_user = dict()
        self.test4days_user_item = dict()
        self.test10days_item_user = dict()
        self.test10days_user_item = dict()
        self.star_items = []
        self.mid_items = []
        self.tail_items = []

    def read_data(self, dirpath):
        with open(dirpath + "/item_user.txt") as iufile:  # list of all items purchased by users
            for line in iufile:
                toks = line.strip().split("\t")
                if len(toks) == 2:
                    i, u = toks[0], toks[1]
                    if i not in self.item_user:
                        self.item_user[i] = []
                    self.item_user[i].append(u)   # {itemid1: [uid1,uid2,...], itemid2: [uid2,uid3,....]} = dict of list of users purchasing each item 
                    if u not in self.user_item:
                        self.user_item[u] = []
                    self.user_item[u].append(i)   # {uid1: [itemid1,itemid2,...], uid2: [itemd2,itemd3,....]} = dict of list of items purchased by each user

        with open(dirpath + "/test2days_item_user.txt") as iufile:  # list of all items purchased by users in 2 days of testing period
            for line in iufile:
                toks = line.strip().split("\t")
                if len(toks) == 2:
                    i, u = toks[0], toks[1]
                    if i not in self.test2days_item_user:
                        self.test2days_item_user[i] = []
                    self.test2days_item_user[i].append(u)   # {itemid1: [uid1,uid2,...], itemid2: [uid2,uid3,....]} = dict of list of users purchasing each item 
                    if u not in self.test2days_user_item:
                        self.test2days_user_item[u] = []
                    self.test2days_user_item[u].append(i)   # {uid1: [itemid1,itemid2,...], uid2: [itemd2,itemd3,....]} = dict of list of items purchased by each user

        with open(dirpath + "/test4days_item_user.txt") as iufile:  # list of all items purchased by users in 4 days of testing period
            for line in iufile:
                toks = line.strip().split("\t")
                if len(toks) == 2:
                    i, u = toks[0], toks[1]
                    if i not in self.test4days_item_user:
                        self.test4days_item_user[i] = []
                    self.test4days_item_user[i].append(u)   # {itemid1: [uid1,uid2,...], itemid2: [uid2,uid3,....]} = dict of list of users purchasing each item 
                    if u not in self.test4days_user_item:
                        self.test4days_user_item[u] = []
                    self.test4days_user_item[u].append(i)   # {uid1: [itemid1,itemid2,...], uid2: [itemd2,itemd3,....]} = dict of list of items purchased by each user

        with open(dirpath + "/test10days_item_user.txt") as iufile:  # list of all items purchased by users in 10 days of testing period
            for line in iufile:
                toks = line.strip().split("\t")
                if len(toks) == 2:
                    i, u = toks[0], toks[1]
                    if i not in self.test10days_item_user:
                        self.test10days_item_user[i] = []
                    self.test10days_item_user[i].append(u)   # {itemid1: [uid1,uid2,...], itemid2: [uid2,uid3,....]} = dict of list of users purchasing each item 
                    if u not in self.test10days_user_item:
                        self.test10days_user_item[u] = []
                    self.test10days_user_item[u].append(i)   # {uid1: [itemid1,itemid2,...], uid2: [itemd2,itemd3,....]} = dict of list of items purchased by each user

        
        with open(dirpath + "/user_friend.txt") as uffile:  # list of all friendship linkages
            for line in uffile:
                toks = line.strip().split("\t")
                if len(toks) == 2:
                    u, f = toks[0], toks[1]
                    if u not in self.user_friend:
                        self.user_friend[u] = []
                    self.user_friend[u].append(f)   # {uid1: [friendid1,friendid2,...], uid2: [friendid2,friendid3,....]} = dict of each person's friends
        
        with open(dirpath + "/selected_user.txt") as sufile:  # pull the randomly selected users
            for user in sufile:
                self.userlist.append(user.strip())
                
    def make_user_recommendations_selected_cluster(self, cluster_items):
        top_5_item_recusers = dict()
        top_10_item_recusers = dict()
        top_15_item_recusers = dict()
        top_20_item_recusers = dict()
        for item in cluster_items:
            user_simscore = {}
            temp_user_simscore = dict()
            users = self.item_user.get(item,[])
            if (users != []):
                for user1 in users:
                    items1 = self.user_item.get(user1,[])
                    friends1 = set(self.user_friend.get(user1,[]))
                    if len(friends1) > 0:
                        for user2 in self.userlist:
                            items2 = self.user_item.get(user2,[])
                            if (user1 == user2): continue
                            if item not in items2:
                                friends2 = set(self.user_friend.get(user2,[]))
                                mutual_friends = len(friends1 & friends2)
                                total_friends = len(friends1 | friends2)
                                friend_similarity_score = mutual_friends/total_friends
                                if user2 not in temp_user_simscore:
                                    temp_user_simscore[user2] = []
                                temp_user_simscore[user2].append(friend_similarity_score)
                for user in temp_user_simscore:
                    user_sim_values = temp_user_simscore[user]
                    user_simscore[user] = max(user_sim_values)
            sorted_user_simscore = sorted(user_simscore.items(), key=lambda x: x[1], reverse = True)
            top_5_item_recusers[item] = [user for user,_ in sorted_user_simscore[0:5]]
            top_10_item_recusers[item] = [user for user,_ in sorted_user_simscore[0:10]]
            top_15_item_recusers[item] = [user for user,_ in sorted_user_simscore[0:15]]
            top_20_item_recusers[item] = [user for user,_ in sorted_user_simscore[0:20]]
        sample_users = {k: top_5_item_recusers[k] for k in list(top_5_item_recusers)[:20]}
        #print(sample_users)
        #print("\n")
        sample_users = {k: top_10_item_recusers[k] for k in list(top_10_item_recusers)[:20]}
        #print(sample_users)
        #print("\n")
        sample_users = {k: top_15_item_recusers[k] for k in list(top_15_item_recusers)[:20]}
        #print(sample_users)
        #print("\n")
        print("Accuracy for Friendship-based Recommender System based on Top-5 similar friends")
        self.get_accuracy_score(item_recusers = top_5_item_recusers, cluster_items = cluster_items)
        print("Accuracy for Friendship-based Recommender System based on Top-10 similar friends")
        self.get_accuracy_score(item_recusers = top_10_item_recusers, cluster_items = cluster_items)
        print("Accuracy for Friendship-based Recommender System based on Top-15 similar friends")
        self.get_accuracy_score(item_recusers = top_15_item_recusers, cluster_items = cluster_items)
        print("Accuracy for Friendship-based Recommender System based on Top-20 similar friends")
        self.get_accuracy_score(item_recusers = top_20_item_recusers, cluster_items = cluster_items)
    
        
    def get_accuracy_score(self, item_recusers, cluster_items):
        count_2days = count_4days = count_10days = 0
        for item in cluster_items:
            counter_2days = counter_4days = counter_10days = 0
            recommended_users = item_recusers.get(item,[])
            for user in recommended_users:
                new_items_2days = self.test2days_user_item.get(user,[])
                new_items_4days = self.test4days_user_item.get(user,[])
                new_items_10days = self.test10days_user_item.get(user,[])
                if item in new_items_2days:
                    counter_2days += 1
                if item in new_items_4days:
                    counter_4days += 1
                if item in new_items_10days:
                    counter_10days += 1        
            if (counter_2days > 0):
                count_2days += 1
            if (counter_4days > 0):
                count_4days += 1
            if (counter_10days > 0):
                count_10days += 1
        print("For a 2-day testing period:", count_2days/len(cluster_items))
        print("For a 4-day testing period:", count_4days/len(cluster_items))
        print("For a 10-day testing period:", count_10days/len(cluster_items))
        
    
    def make_item_recommendations(self):
        with open(dirpath + "/rank_item_quantity_price_sales.txt") as itemfile:  # pull the ranked items, and create clusters
            for line in itemfile:
                toks = line.strip().split("\t")
                if len(toks) == 5:
                    rank = int(toks[0])
                    if rank <= 40:
                        self.star_items.append(toks[1])
                    elif rank > 40 and rank <= 98:
                        self.mid_items.append(toks[1])
                    else:
                        self.tail_items.append(toks[1])
        print(self.star_items)
        print(self.mid_items)
        print(self.tail_items)
        print("\nMaking Recommendations\n")
        print("\nAccuracy for Star Items\n")
        self.make_user_recommendations_selected_cluster(cluster_items = self.star_items)
        print("\nAccuracy for Mid Items\n")
        self.make_user_recommendations_selected_cluster(cluster_items = self.mid_items)
        print("\nAccuracy for Long Tail Items\n")
        self.make_user_recommendations_selected_cluster(cluster_items = self.tail_items)   

                            
dirpath = 'C:/Vidit/PhD/RA Work/KZ - RA/Gaming Project/data/Files'


def main():
    fng = FriendNetworkGenerator()
    fng.read_data(dirpath)
    fng.make_item_recommendations()
    print("\nSuccess")


if __name__ == "__main__":
    main()

['lolvn002', 'lolvn001', 'lolvn003', 'loltw001', 'RP003', 'loltw003', 'RP002', 'loltw004', 'lolvn004', 'RP001', 'package_20', 'loltw005', 'tw_18_3', '89', 'loltw002', 'tw_18_2', 'tw_18_5', 'lolsam001', '9002', 'lolsam003', 'fo3vn003', 'RP004', 'FO3TH003', 'fo3vn004', 'tw_18_4', 'package_4', '9001', 'package_2', 'fo3vn002', 'lolsam004', 'tw_19_3', 'package_18', 'package_19', 'package_1', 'tw_19_2', '166', '145', 'fo3vn005', '9000', '33']
['43', '31', 'RP005', '237', 'HONLUCK1', '57', '2', 'lolsam006', 'lolsam002', '239', 'FO3TH004', 'FO3TH006', 'tw_19_5', 'lolvn005', '157', 'pbth001', '159', '205', 'lolth004', '228', '280', 'tw_19_4', 'lolth001', 'lolsam005', '138', 'lolth003', 'tw_18_1', 'tw_15_5', '34', 'tw_15_4', '188', '25', '192', '200', 'FO3TH005', 'FO3TH001', '230', 'elph003', 'lolth005', 'fo3sam001', '12', 'fo3vn001', 'lolth002', 'FO3TH007', 'tw_15_3', 'tw_15_1', '26', '278', '95', '76', 'pbth003', 'pbth004', 'lolsam008', '244', '151', '14', 'elph002', 'loltw006']
['RP006', 'pbt

In [3]:
class FriendNetworkGenerator:
    def __init__(self):
        self.user_friend = dict()
        self.item_user = dict()
        self.user_item = dict()
        self.userlist = []
        self.test2days_item_user = dict()
        self.test2days_user_item = dict()
        self.test4days_item_user = dict()
        self.test4days_user_item = dict()
        self.test10days_item_user = dict()
        self.test10days_user_item = dict()
        self.star_items = []
        self.mid_items = []
        self.tail_items = []
        self.all_items = []

    def read_data(self, dirpath):
        with open(dirpath + "/item_user.txt") as iufile:  # list of all items purchased by users
            for line in iufile:
                toks = line.strip().split("\t")
                if len(toks) == 2:
                    i, u = toks[0], toks[1]
                    if i not in self.item_user:
                        self.item_user[i] = []
                    self.item_user[i].append(u)   # {itemid1: [uid1,uid2,...], itemid2: [uid2,uid3,....]} = dict of list of users purchasing each item 
                    if u not in self.user_item:
                        self.user_item[u] = []
                    self.user_item[u].append(i)   # {uid1: [itemid1,itemid2,...], uid2: [itemd2,itemd3,....]} = dict of list of items purchased by each user

        with open(dirpath + "/test2days_item_user.txt") as iufile:  # list of all items purchased by users in 2 days of testing period
            for line in iufile:
                toks = line.strip().split("\t")
                if len(toks) == 2:
                    i, u = toks[0], toks[1]
                    if i not in self.test2days_item_user:
                        self.test2days_item_user[i] = []
                    self.test2days_item_user[i].append(u)   # {itemid1: [uid1,uid2,...], itemid2: [uid2,uid3,....]} = dict of list of users purchasing each item 
                    if u not in self.test2days_user_item:
                        self.test2days_user_item[u] = []
                    self.test2days_user_item[u].append(i)   # {uid1: [itemid1,itemid2,...], uid2: [itemd2,itemd3,....]} = dict of list of items purchased by each user

        with open(dirpath + "/test4days_item_user.txt") as iufile:  # list of all items purchased by users in 4 days of testing period
            for line in iufile:
                toks = line.strip().split("\t")
                if len(toks) == 2:
                    i, u = toks[0], toks[1]
                    if i not in self.test4days_item_user:
                        self.test4days_item_user[i] = []
                    self.test4days_item_user[i].append(u)   # {itemid1: [uid1,uid2,...], itemid2: [uid2,uid3,....]} = dict of list of users purchasing each item 
                    if u not in self.test4days_user_item:
                        self.test4days_user_item[u] = []
                    self.test4days_user_item[u].append(i)   # {uid1: [itemid1,itemid2,...], uid2: [itemd2,itemd3,....]} = dict of list of items purchased by each user

        with open(dirpath + "/test10days_item_user.txt") as iufile:  # list of all items purchased by users in 10 days of testing period
            for line in iufile:
                toks = line.strip().split("\t")
                if len(toks) == 2:
                    i, u = toks[0], toks[1]
                    if i not in self.test10days_item_user:
                        self.test10days_item_user[i] = []
                    self.test10days_item_user[i].append(u)   # {itemid1: [uid1,uid2,...], itemid2: [uid2,uid3,....]} = dict of list of users purchasing each item 
                    if u not in self.test10days_user_item:
                        self.test10days_user_item[u] = []
                    self.test10days_user_item[u].append(i)   # {uid1: [itemid1,itemid2,...], uid2: [itemd2,itemd3,....]} = dict of list of items purchased by each user

        
        with open(dirpath + "/user_friend.txt") as uffile:  # list of all friendship linkages
            for line in uffile:
                toks = line.strip().split("\t")
                if len(toks) == 2:
                    u, f = toks[0], toks[1]
                    if u not in self.user_friend:
                        self.user_friend[u] = []
                    self.user_friend[u].append(f)   # {uid1: [friendid1,friendid2,...], uid2: [friendid2,friendid3,....]} = dict of each person's friends
        
        with open(dirpath + "/selected_user.txt") as sufile:  # pull the randomly selected users
            for user in sufile:
                self.userlist.append(user.strip())
                
    def make_user_recommendations_selected_cluster(self, cluster_items):
        top_5_item_recusers = dict()
        top_10_item_recusers = dict()
        top_15_item_recusers = dict() 
        top_20_item_recusers = dict() 
        for item in cluster_items:
            user_simscore = {}
            temp_user_simscore = dict()
            users = self.item_user.get(item,[])
            if (users != []):
                for user1 in users:
                    items1 = self.user_item.get(user1,[])
                    friends1 = set(self.user_friend.get(user1,[]))
                    if len(friends1) > 0:
                        for friend in friends1:
                            items2 = self.user_item.get(friend,[])
                            if item not in items2:
                                friends2 = set(self.user_friend.get(friend,[]))
                                mutual_friends = len(friends1 & friends2)
                                total_friends = len(friends1 | friends2)
                                friend_similarity_score = mutual_friends/total_friends
                                if friend not in temp_user_simscore:
                                    temp_user_simscore[friend] = []
                                temp_user_simscore[friend].append(friend_similarity_score)
                for user in temp_user_simscore:
                    user_sim_values = temp_user_simscore[user]
                    user_simscore[user] = max(user_sim_values)
            sorted_user_simscore = sorted(user_simscore.items(), key=lambda x: x[1], reverse = True)
            top_5_item_recusers[item] = [user for user,_ in sorted_user_simscore[0:5]]
            top_10_item_recusers[item] = [user for user,_ in sorted_user_simscore[0:10]]
            top_15_item_recusers[item] = [user for user,_ in sorted_user_simscore[0:15]]
            top_20_item_recusers[item] = [user for user,_ in sorted_user_simscore[0:20]]
        print("Accuracy for Friendship-based Recommender System based on Top-5 similar friends")
        self.get_accuracy_score(item_recusers = top_5_item_recusers, cluster_items = cluster_items)
        print("\n")
        print("Accuracy for Friendship-based Recommender System based on Top-10 similar friends")
        self.get_accuracy_score(item_recusers = top_10_item_recusers, cluster_items = cluster_items)
        print("\n")
        print("Accuracy for Friendship-based Recommender System based on Top-15 similar friends")
        self.get_accuracy_score(item_recusers = top_15_item_recusers, cluster_items = cluster_items)
        print("\n")
        print("Accuracy for Friendship-based Recommender System based on Top-20 similar friends")
        self.get_accuracy_score(item_recusers = top_20_item_recusers, cluster_items = cluster_items)
    
        
    def get_accuracy_score(self, item_recusers, cluster_items):
        count_2days = count_4days = count_10days = 0
        for item in cluster_items:
            counter_2days = counter_4days = counter_10days = 0
            recommended_users = item_recusers.get(item,[])
            for user in recommended_users:
                new_items_2days = self.test2days_user_item.get(user,[])
                new_items_4days = self.test4days_user_item.get(user,[])
                new_items_10days = self.test10days_user_item.get(user,[])
                if item in new_items_2days:
                    counter_2days += 1
                if item in new_items_4days:
                    counter_4days += 1
                if item in new_items_10days:
                    counter_10days += 1        
            if (counter_2days > 0):
                count_2days += 1
            if (counter_4days > 0):
                count_4days += 1
            if (counter_10days > 0):
                count_10days += 1
        print("For a 2-day testing period:", count_2days/len(cluster_items))
        print("For a 4-day testing period:", count_4days/len(cluster_items))
        print("For a 10-day testing period:", count_10days/len(cluster_items))
        
    
    def make_item_recommendations(self):
        with open(dirpath + "/rank_item_quantity_price_sales.txt") as itemfile:  # pull the ranked items, and create clusters
            for line in itemfile:
                toks = line.strip().split("\t")
                if len(toks) == 5:
                    self.all_items.append(toks[1])
                    rank = int(toks[0])
                    if rank <= 40:
                        self.star_items.append(toks[1])
                    else:
                        self.tail_items.append(toks[1])
        #print(self.star_items)
        #print(self.tail_items)
        #print(self.all_items)
        print("\nMaking Recommendations\n")
        print("\nAccuracy for Star Items\n")
        self.make_user_recommendations_selected_cluster(cluster_items = self.star_items)
        print("\nAccuracy for Long Tail Items\n")
        self.make_user_recommendations_selected_cluster(cluster_items = self.tail_items)
        print("\nAccuracy for All Items\n")
        self.make_user_recommendations_selected_cluster(cluster_items = self.all_items)

                            
dirpath = 'C:/Vidit/PhD/RA Work/KZ - RA/Gaming Project/data/Files'


def main():
    fng = FriendNetworkGenerator()
    fng.read_data(dirpath)
    fng.make_item_recommendations()
    print("\nSuccess")


if __name__ == "__main__":
    main()


Making Recommendations


Accuracy for Star Items

Accuracy for Friendship-based Recommender System based on Top-5 similar friends
For a 2-day testing period: 0.05
For a 4-day testing period: 0.05
For a 10-day testing period: 0.05


Accuracy for Friendship-based Recommender System based on Top-10 similar friends
For a 2-day testing period: 0.05
For a 4-day testing period: 0.125
For a 10-day testing period: 0.15


Accuracy for Friendship-based Recommender System based on Top-15 similar friends
For a 2-day testing period: 0.075
For a 4-day testing period: 0.15
For a 10-day testing period: 0.2


Accuracy for Friendship-based Recommender System based on Top-20 similar friends
For a 2-day testing period: 0.075
For a 4-day testing period: 0.15
For a 10-day testing period: 0.3

Accuracy for Long Tail Items

Accuracy for Friendship-based Recommender System based on Top-5 similar friends
For a 2-day testing period: 0.004484304932735426
For a 4-day testing period: 0.008968609865470852
For a 10-d